## setup

In [15]:
if not "cdSet" in globals():
    %cd -q ..
    cdSet = True

In [16]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import models.networks as networks
# from collections import OrderedDict
from data.cityscapes import Cityscapes
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from PIL import Image
import matplotlib.pylab as plt

In [66]:
variational_beta = 0.0001
num_epochs = 10
latent_dims = 1024
batch_size = 8
learning_rate = 1e-5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
H, W = 512, 512

mapping = {
    (0, 0, 0): 0,
    (0, 0, 0): 1,
    (0, 0, 0): 2,
    (0, 0, 0): 3,
    (0, 0, 0): 4,
    (111, 74, 0): 5,
    (81, 0, 81): 6,
    (128, 64, 128): 7,
    (244, 35, 232): 8,
    (250, 170, 160): 9,
    (230, 150, 140): 10,
    (70, 70, 70): 11,
    (102, 102, 156): 12,
    (190, 153, 153): 13,
    (180, 165, 180): 14,
    (150, 100, 100): 15,
    (150, 120, 90): 16,
    (153, 153, 153): 17,
    (153, 153, 153): 18,
    (250, 170, 30): 19,
    (220, 220, 0): 20,
    (107, 142, 35): 21,
    (152, 251, 152): 22,
    (70, 130, 180): 23,
    (220, 20, 60): 24,
    (255, 0, 0): 25,
    (0, 0, 142): 26,
    (0, 0, 70): 27,
    (0, 60, 100): 28,
    (0, 0, 90): 29,
    (0, 0, 110): 30,
    (0, 80, 100): 31,
    (0, 0, 230): 32,
    (119, 11, 32): 33,
    (0, 0, 142): 34,
}

In [67]:
train_loss_avg = []
val_loss_avg = []
n_iter = 0
num_batches = 0
train_loss_avg.append(0)
val_loss_avg.append(0)
pixel_size = batch_size * H * W

In [68]:
dataset_train = Cityscapes(
    root="../../maskgan/data/cityscapes/",
    split="train",
    mode="fine",
    target_type="semantic",
    transform=None,
    target_transform=None,
    transforms=None,
)

In [74]:
!nvidia-smi

Sun Nov 29 11:27:46 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:18:00.0 Off |                    0 |
| N/A   38C    P0    68W / 300W |  27750MiB / 32510MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:3B:00.0 Off |                    0 |
| N/A   34C    P0    78W / 300W |  24588MiB / 32510MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [ ]:
!kill -9 209188